In [1]:
import tensorflow as tf

model = tf.keras.models.load_model('models\converted_keras (4)\keras_model.h5')

In [2]:
import cv2

cap = cv2.VideoCapture(0)

In [3]:
def preprocess(frame):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Resize to match model input shape
    resized = cv2.resize(gray, (224, 224))
    
    # Convert to 3-channel image
    processed_frame = cv2.cvtColor(resized, cv2.COLOR_GRAY2BGR)
    
    # Normalize pixel values
    processed_frame = processed_frame / 255.0
    
    # Add batch dimension
    processed_frame = processed_frame[np.newaxis, ...]
    
    return processed_frame


In [4]:
def get_class_label(class_index):
    class_labels = {
        0: 'peace',
        1: 'No',
        2: 'Father',
        3: 'I love you',
        4: 'IDLE NOTHING',
        5: 'Good Luck',
    }
    return class_labels[class_index]

In [5]:
import numpy as np

def postprocess(frame, predictions):
    # Extract the predicted class
    predicted_class = np.argmax(predictions)
    
    # Get the class label
    class_label = get_class_label(predicted_class)
    
    # Draw the class label on the frame
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    thickness = 2
    color = (0, 255, 0)  # green
    text_size = cv2.getTextSize(class_label, font, font_scale, thickness)[0]
    text_x = int((frame.shape[1] - text_size[0]) / 2)
    text_y = int((frame.shape[0] + 300 + text_size[1]) / 2)
    cv2.putText(frame, class_label, (text_x, text_y), font, font_scale, color, thickness)
    
    return frame


In [6]:
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Preprocess the frame
    processed_frame = preprocess(frame)
    
    # Use the model to make predictions
    predictions = model.predict(processed_frame)
    
    # Postprocess the predictions and overlay on the frame
    overlay = postprocess(frame, predictions)
    
    # Display the resulting frame
    cv2.imshow('frame', overlay)
    
    # Check for exit key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

1/1 [==============================] - 0s 38ms/step


In [7]:
cap.release()
cv2.destroyAllWindows()